# Dataset

CSV file for the bitcoin price for the time period of 01.01.2012 10:01 until 13.04.2025 02:13.

In [ ]:
import os
from kaggle.api.kaggle_api_extended import KaggleApi
from zipfile import ZipFile

ZIP_PATH = "data/zip/"
ZIP_NAME = "bitcoin-historical-data.zip"
ZIP_FILE = os.path.join(ZIP_PATH, ZIP_NAME)

RAW_PATH = "data/raw/"
RAW_NAME = "btcusd_1-min_data.csv"
RAW_FILE = os.path.join(RAW_PATH, RAW_NAME)

# Validate folders exist
os.makedirs(ZIP_PATH, exist_ok=True)
os.makedirs(RAW_PATH, exist_ok=True)

api = KaggleApi()
api.authenticate()
api.dataset_download_files("mczielinski/bitcoin-historical-data", path=ZIP_PATH)

# Extract zip
zip_file = ZipFile(ZIP_FILE)
zip_file.extractall(path=RAW_PATH)

# Remove zip
os.remove(ZIP_FILE)

In [ ]:
# Process dataset

import os
import pytz
import shutil
from datetime import datetime
import pandas as pd

PROCESSED_PATH = "data/processed/"
PROCESSED_NAME = "btcusd_1-min_data_processed.csv"
PROCESSED_FILE = os.path.join(PROCESSED_PATH, PROCESSED_NAME)

# Validate folders exist
os.makedirs(PROCESSED_PATH, exist_ok=True)


# Function to convert Unix timestamp to formatted datetime string
def convert_timestamp(ts):
    dt = datetime.fromtimestamp(float(ts), tz=pytz.UTC)
    timezone_str = dt.strftime("%z")
    formatted_timezone = f"{timezone_str[:3]}:{timezone_str[3:]}"
    return dt.strftime("%Y-%m-%d %H:%M:%S") + formatted_timezone


# Copy the original file to processed directory
shutil.copy2(RAW_FILE, PROCESSED_FILE)

# Convert Timestamp column to datetime format
df = pd.read_csv(PROCESSED_FILE, low_memory=False)
df["datetime"] = df["Timestamp"].apply(convert_timestamp)

# Save the processed data
df.to_csv(PROCESSED_FILE, index=False)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns


def plot_missing_values(df, figsize=(12, 8)):
    # Create a figure with two subplots
    fig, axes = plt.subplots(2, 1, figsize=figsize)

    # Heatmap of missing values
    sns.heatmap(df.isna(), cmap="viridis", yticklabels=False, cbar=False, ax=axes[0])
    axes[0].set_title("Missing Values Heatmap")
    axes[0].set_xlabel("Variables")
    axes[0].set_ylabel("Observations")

    # Percentage of missing values per column
    missing_percentage = df.isna().sum() / len(df) * 100
    missing_percentage.sort_values(ascending=False).plot(kind="bar", ax=axes[1])
    axes[1].set_title("Percentage of Missing Values per Column")
    axes[1].set_xlabel("Columns")
    axes[1].set_ylabel("Percentage of Missing Values")
    axes[1].grid(axis="y", linestyle="--", alpha=0.7)

    # Space between subplots
    plt.tight_layout()
    plt.show()

    # Print summary of missing values
    missing_values = df.isna().sum()
    missing_percent = missing_values / len(df) * 100
    summary = pd.DataFrame(
        {"Missing Values": missing_values, "Percentage": missing_percent}
    )
    # Only show columns with missing values
    summary = summary[summary["Missing Values"] > 0].sort_values(
        "Percentage", ascending=False
    )

    if len(summary) > 0:
        print("Missing Values Summary:")
        print(summary)
    else:
        print("No missing values found in the dataset.")

    return summary

In [ ]:
import os
import pandas as pd

PROCESSED_PATH = "data/processed/"
PROCESSED_NAME = "btcusd_1-min_data_processed.csv"
PROCESSED_FILE = os.path.join(PROCESSED_PATH, PROCESSED_NAME)

df = pd.read_csv(PROCESSED_FILE)
df["Timestamp"] = pd.to_datetime(df["Timestamp"], unit="s", utc=True)
df.set_index("Timestamp", inplace=True)
df.drop(columns=["datetime"], inplace=True)
df.info()

summary = plot_missing_values(df)

pd.set_option("display.precision", 2)
df.describe()